In [ ]:
import boost_histogram as bh
import numpy as np
import matplotlib.pyplot as plt

## 5: Understanding accumulators

Boost-histogram has several different storages; storages store accumulators. Let's try making a profile.

In [ ]:
mean = bh.accumulators.Mean()
mean.fill([.3, .4, .5])

Here's a quick example accessing the values:

In [ ]:
print(f"mean.count={mean.count} mean.value={mean.value:g} mean.variance={mean.variance:g}")

# Python 3.8:
# print(f"{mean.count=} {mean.value=} {mean.variance=}")

## 6: Changing the storage

In [ ]:
hist6 = bh.Histogram(bh.axis.Regular(10, 0, 10), storage=bh.storage.Mean())

In [ ]:
hist6.fill([0.5]*3, sample=[.3, .4, .5])

In [ ]:
hist6[0]

In [ ]:
hist6.view()

In [ ]:
hist6.view().value

In [ ]:
hist6.view().variance

## 7: Making a density histogram

Let's try to make a density histogram like Numpy's.

In [ ]:
bins = [-10, -7, -4, -3, -2, -1, -.75, -.5, -.25, 0, .25, .5, .75, 1, 2, 3, 4, 7, 10]
d7, e7 = np.histogram(data1 - 3.5, bins=bins, density=True)
plt.hist(data1 - 3.5, bins=bins, density=True);

Yes, it's ugly. Don't judge.

We don't have a `.density`! What do we do? (note: `density=True` is supported if you do not return a bh object)

In [ ]:
hist7 = bh.numpy.histogram(data1 - 3.5, bins=bins, histogram=bh.Histogram)

widths = hist7.axes.widths
area = np.prod(widths, axis=0)

area

Yes, that does not need to be so complicated for 1D, but it's general.

In [ ]:
factor = np.sum(hist7.view())
view = hist7.view() / (factor * area)

In [ ]:
plt.bar(hist7.axes[0].centers, view, width=hist7.axes[0].widths);

## 8: Axis types

There are more axes types, and they all provide the same API in histograms, so they all just work without changes:

In [ ]:
hist8 = bh.Histogram(
    bh.axis.Regular(30, 1, 10, transform=bh.axis.transform.log),
    bh.axis.Regular(30, 1, 10, transform=bh.axis.transform.sqrt)
)

In [ ]:
hist8.reset()
hist8.fill(*make_2D_data(mean=(5, 5), widths=(5, 5)))

In [ ]:
plothist2d(hist8);

## 9: And, circular, too!

In [ ]:
hist9 = bh.Histogram(bh.axis.Regular(30, 0, 2*np.pi, circular=True))
hist9.fill(np.random.uniform(0, np.pi*4, size=300))

Now, the really complicated part, making a circular histogram:


In [ ]:
ax = plt.subplot(111, polar=True)
plothist(hist9);